# Apresentação da classe `Receptor`

In [113]:
import socket
import pickle
from threading import Thread

## Importações

1. `socket`: Este módulo permite a comunicação em rede. Ele fornece a funcionalidade de soquetes de rede, que são os pontos finais de uma conexão de comunicação.

2. `pickle`: Este módulo é usado para serializar e desserializar objetos Python. A serialização é o processo de converter um objeto em um formato que pode ser transmitido ou armazenado, e a desserialização é o processo inverso.

3. `Thread` de `threading`: Este módulo é usado para criar threads, que são fluxos de execução separados. Isso pode ser útil para fazer várias coisas ao mesmo tempo.

Obs.: Para uma maior legibilidade, esconda os detalhes de implementação na célula seguinte.

In [114]:
class Receiver:
    def __init__(self, host='127.0.0.1', port=65432):
        self.host = host
        self.port = port
        self.running = True
        self.bits_array = []
        self.server_thread: Thread

    def __binary_2_text(self, bits):
        """ Converts binary to text """
        bits_str = ''.join(map(str, bits))  # convert list of bits to a string of bits
        bytes_list = [bits_str[i:i+8] for i in range(0, len(bits_str), 8)]  # divide the string of bits into bytes
        bytes_list = [int(byte, 2) for byte in bytes_list]  # convert bytes to integers
        bytes_array = bytearray(bytes_list)  # convert list of integers to bytearray
        text = bytes_array.decode('utf8')  # decode bytearray to string
        return text

    def start_server(self):
        self.server_thread = Thread(target=self._start_server)
        self.server_thread.daemon = True
        self.server_thread.start()

    def _start_server(self):
        socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        socket_servidor.bind((self.host, self.port))
        socket_servidor.listen(1)
        print("Listening on port 65432")
        while True and self.running:
            conexao_socket, end = socket_servidor.accept()
            print(end, 'Connected!')

            dados = conexao_socket.recv(4096)  # receber ate 1024 bytes
            dados_list = pickle.loads(dados)
            self.bits_array = dados_list

            conexao_socket.send(pickle.dumps(self.bits_array))

            conexao_socket.close()


# Run methods start ---------------------------------------------------------------------------------------------------------------------

    def run(self, encoding_method, framing_method, error_correction_or_detection_method):

        match framing_method.lower():
            case "character_count":
                self.frames, self.padding_bits_list = self.character_count_deframing(self.bits_array)
            case "byte_insertion":
                self.frames, self.padding_bits_list  = self.bytes_insertion_deframing(self.bits_array)
            case "bits_insertion":
                if error_correction_or_detection_method == "crc":
                    self.frames, self.padding_bits_list  = self.bits_insertion_deframing(self.bits_array, crc32=True)
                else:
                    self.frames, self.padding_bits_list  = self.bits_insertion_deframing(self.bits_array, crc32=False)

        match error_correction_or_detection_method.lower():
            case "even_parity":
                self.bits_cleaned, self.list_error_detec = self.solve_even_parity(self.frames, self.padding_bits_list)
            case "crc":
                self.bits_cleaned, self.list_error_detec = self.solve_crc32(self.frames, self.padding_bits_list)
            case "hamming":
                self.bits_cleaned, self.list_error_detec = self.solve_hamming(self.frames, self.padding_bits_list)


        match encoding_method.lower():
            case "nrz":	# -1 -> 0; 1 -> 1
                pass
            case "bipolar":	# 0 -> 0; (-1,1) -> 1
                pass
            case "manchester":	# 0 -> 0; 1 -> 1
                bit_pairs = [self.bits_cleaned[i:i+2] for i in range(0, len(self.bits_cleaned), 2)]
                self.bits_cleaned = [0 if pair == [0, 1] else 1 for pair in bit_pairs]
                
        
        final_str = self.__binary_2_text(self.bits_cleaned)

        bits_cleaned_str = ''.join(map(str, self.bits_cleaned))

        return self.bits_array, bits_cleaned_str, final_str

# Run methods end ---------------------------------------------------------------------------------------------------------------------



# Framing methods start ---------------------------------------------------------------------------------------------------------------------

    def character_count_deframing(self, bits_array):
        """Return a matrix of frames without headers"""
        original_frames_matrix = []
        padding_bits_list = []
        bytes_list = [''.join(map(str, bits_array[i:i+8])) for i in range(0, len(bits_array), 8)] # bytes_list is a array of strings of 8 bits

        while bytes_list:
            # Convert the header to integer
            frame_size = int(bytes_list[0], 2)

            padding_bits = int(bytes_list[1], 2)
            # Remove the header and add the frame to the matrix
            original_frames_matrix.append(bytes_list[2:frame_size])
            padding_bits_list.append(padding_bits)
            # Move to the next frame
            bytes_list = bytes_list[frame_size:]

        list_str_frames = [''.join(frame) for frame in original_frames_matrix]

        return list_str_frames, padding_bits_list
    

    def bytes_insertion_deframing(self, bits_array):
        """Return a matrix of frames without flags"""
        original_frames_matrix = []
        frame = []
        padding_bits_list = []
        bytes_list = [''.join(map(str, bits_array[i:i+8])) for i in range(0, len(bits_array), 8)] # bytes_list is a array of strings of 8 bits
        flag = '01111110'

        while bytes_list:
            byte = bytes_list.pop(0)
            if byte == flag:
                if frame:  # if frame is not empty
                    original_frames_matrix.append(frame[1:])
                    padding_bits = int(frame[0], 2)
                    padding_bits_list.append(padding_bits)
                    frame = []
            else:
                frame.append(byte)

        list_str_frames = [''.join(frame) for frame in original_frames_matrix]

        return list_str_frames, padding_bits_list

    
    def bits_insertion_deframing(self, bits_array, crc32=False): 
        """Return a list of frames(str) without flags"""
        original_frames_list = []
        frame = []
        padding_bits_list = []
        bits_string = ''.join(map(str, bits_array)) # bits_string is a string of bits
        flag = '01111110'

        while bits_string:
            if bits_string.startswith(flag):
                bits_string = bits_string[len(flag):]  # remove the flag
                if crc32 and not frame:
                    padding_bits = int(bits_string[:8], 2)
                    bits_string = bits_string[8:]
                    padding_bits_list.append(padding_bits)
                elif not crc32 and not frame:
                    padding_bits_list.append(0)

                if frame:  # if frame is not empty
                    original_frames_list.append("".join(frame))
                    frame = []
            else:
                frame.append(bits_string[0])
                bits_string = bits_string[1:]

        return original_frames_list, padding_bits_list

# Framing methods end ---------------------------------------------------------------------------------------------------------------------
    



# Error correction or detection methods start ---------------------------------------------------------------------------------------------------------------------
    
    def solve_even_parity(self, frames, padding_bits_list):
        list_detection_error = []
        list_bits_cleaned = []

        for frame, padding_bits in zip(frames, padding_bits_list):
            if padding_bits != 0:
                frame = frame[:-padding_bits] # remove the padding bits

            bits_array = [int(bit) for bit in frame] # convert the frame to a list of integers
            count_ones = sum(bits_array[:-1])
            parity_bit = bits_array[-1]

            if (count_ones + parity_bit) % 2 == 0:
                list_detection_error.append(False)
            else:
                list_detection_error.append(True)	

            list_bits_cleaned.extend(bits_array[:-1])		

        return list_bits_cleaned, list_detection_error



    def solve_crc32(self, frames, padding_bits_list):
        def verify_crc32(bit_array):
            inserted_bits_len = 0
            crc32_polynomial = 0x104C11DB7 # polynomial used by CRC32 IEEE 802 (0x04C11DB7 without the occlusion of the first bit)
            crc32_polynomial_str = f"{crc32_polynomial:033b}" # 32 0's to complete the 96 bits

            def xor(bit_str_a, bit_str_b): # xor between two bit strings
                bit_str_result = ''
                for i in range(len(bit_str_a)):
                    if bit_str_a[i] == bit_str_b[i]: # if the bits are equal, the result is 0
                        bit_str_result += '0'
                    else: # if the bits are different, the result is 1
                        bit_str_result += '1'
                return bit_str_result

            bit_str = ''.join(map(str, bit_array)) # convert the bit array to a bit string

            bit_str = bit_str + '0'*32 # 32 0's to complete the 96 bits     
            bit_str_to_xor = ''
            for i in range(len(bit_str)): # for each bit in the bit string
                if i <= 32: # if the bit is in the first 32 bits
                    bit_str_to_xor += bit_str[i]
                else: # if the bit is in the last 64 bits
                    if bit_str_to_xor[0] == '1':
                        bit_str_to_xor = xor(bit_str_to_xor, crc32_polynomial_str) # xor with the polynomial
                        bit_str_to_xor = bit_str_to_xor[1:] + bit_str[i] # exclude the first bit (0) and include the next bit
                    else:
                        bit_str_to_xor = bit_str_to_xor[1:] + bit_str[i] # exclude the first bit (0) and include the next bit

                if i == len(bit_str) - 1: # if it is the last bit
                    if bit_str_to_xor[0] == '1': # if the first bit is 1, xor with the polynomial
                        bit_str_to_xor = xor(bit_str_to_xor, crc32_polynomial_str)
                        bit_str_to_xor = bit_str_to_xor[1:] # exclude the first bit (0)

                    else: # if the first bit is 0, xor with 33 0's
                        bit_str_to_xor = bit_str_to_xor[1:] # exclude the first bit (0)

            if bit_str_to_xor == '0'*32: # if the result is 32 0's, don't have errors
                return True
            else: # if the result is not 32 0's, have errors
                return False

        list_detection_error = []
        list_bits_cleaned = []
        for frame, padding_bits in zip(frames, padding_bits_list):
            bits_array = [int(bit) for bit in frame] # convert the frame to a list of integers
            if verify_crc32(bits_array):
                list_detection_error.append(False)
            else:
                list_detection_error.append(True)
            
            if padding_bits != 0:
                list_bits_cleaned.extend(bits_array[:-padding_bits-32]) # remove the padding bits
            else:
                list_bits_cleaned.extend(bits_array[:-32]) 

        return list_bits_cleaned, list_detection_error
    
    def solve_hamming(self, frames, padding_bits_list): # Apply the Hamming Code to the provided bit array.
        def find_len_redundant_bits(bit_array): 
            """"""
            len_bit_array = len(bit_array)
            i = 0
            while ((2**i) <= len_bit_array):
                i += 1
            return i

        def calculate_parity_bit(bit_array, position): # position must be one of the power of 2 (1, 2, 4, 8, 16, ...)
            """Calculate the parity bit for the given position."""
            temp_bit_array = bit_array[position-1:]
            list_of_bits = []
            jump = False # jump must be started with False to collect the first bits of the bit_array according to the position
            for i in range(0, len(bit_array), position):
                if jump:
                    jump = False
                    continue

                list_of_bits.extend(temp_bit_array[i:i+position]) # if i+position is greater than the length of the temp_bit_array, it will not be a problem because the slice will be until the end of the list
                jump = True        

            parity = list_of_bits[0] # The first bit is the parity bit itself
            list_of_bits = list_of_bits[1:] 

            for bit in list_of_bits:
                parity ^= bit

            return parity
                
        def make_correction(bit_array):
            """Make the correction of the bit array."""
            len_redudant_bits = find_len_redundant_bits(bit_array)
            error_position = 0
            str_bin_correction = ""

            for i in range(len_redudant_bits):
                position = (2**i)
                parity = calculate_parity_bit(bit_array, position)
                str_bin_correction += str(parity)

            str_bin_correction = str_bin_correction[::-1] # Reverse the string

            print("str_bin_correction",str_bin_correction)
            error_position = int(str_bin_correction, 2) # Convert the binary string to decimal

            if error_position == 0:
                print("No error detected")

            else:
                print(f"Error detected at position {error_position}")
                error_position -= 1  # Adjusting for 0-based index

                if bit_array[error_position] == 0:
                    bit_array[error_position] = 1
                else:
                    bit_array[error_position] = 0
            
            bit_array_corrected_cleaned = []
            for i in range(len(bit_array)):
                if (i+1) not in [2**i for i in range(len_redudant_bits)]:
                    bit_array_corrected_cleaned.append(bit_array[i])

            return bit_array_corrected_cleaned
        

        list_detection_error = []
        list_bits_cleaned = []
        for frame, padding_bits in zip(frames, padding_bits_list):
            if padding_bits != 0:
                frame = frame[:-padding_bits] # remove the padding bits

            bits_array = [int(bit) for bit in frame] # convert the frame to a list of integers
            bits_array_corrected = make_correction(bits_array)

            list_bits_cleaned.extend(bits_array_corrected)	
            

        return list_bits_cleaned, list_detection_error

# Error correction or detection methods end ---------------------------------------------------------------------------------------------------------------------



## Construção da classe


```py

def __init__(self, host='127.0.0.1', port=65432):
    self.host = host
    self.port = port
    self.running = True
    self.bits_array = []
    self.server_thread: Thread

def __binary_2_text(self, bits):
    """ Converts binary to text """
    bits_str = ''.join(
        map(str, bits))  # convert list of bits to a string of bits
    # divide the string of bits into bytes
    bytes_list = [bits_str[i:i+8] for i in range(0, len(bits_str), 8)]
    bytes_list = [int(byte, 2)
                    for byte in bytes_list]  # convert bytes to integers
    # convert list of integers to bytearray
    bytes_array = bytearray(bytes_list)
    text = bytes_array.decode('utf8')  # decode bytearray to string

    return text

```

1. `__init__`: Este é o método construtor que é chamado quando um objeto é criado a partir desta classe. Ele inicializa o objeto com os valores fornecidos para `host` e `port`, ou com os valores padrão '127.0.0.1' e 65432 se nenhum valor for fornecido. Ele também define o atributo `running` como True, para indicar que o servidor está rodando e inicializa uma lista vazia `bits_array` - que serão os dados recebidos pelo transmissor - e declara `server_thread` como uma instância de `Thread` para nos permitir criar um servidor ao lado da aplicação.


2. `__binary_2_text`: Este é um método privado que converte uma sequência binária em texto.

Exemplo:

In [115]:
receptor = Receiver()
print("Host:", receptor.host, "Port:", receptor.port)


Host: 127.0.0.1 Port: 65432


## Criação de socket

Para que os dados sejam transmitidos entre o transmissor e o receptor, é necessário criar um socket. Um socket é um ponto final de uma conexão de comunicação bidirecional entre dois programas em uma rede.


```py

def start_server(self):
        self.server_thread = Thread(target=self._start_server)
        self.server_thread.daemon = True
        self.server_thread.start()

def _start_server(self):
    socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socket_servidor.bind((self.host, self.port))
    socket_servidor.listen(1)
    print("Listening on port 65432")
    while True and self.running:
        conexao_socket, end = socket_servidor.accept()
        print(end, 'Connected!')

        dados = conexao_socket.recv(4096)  # receber ate 1024 bytes

        dados_list = pickle.loads(dados)
        self.bits_array = dados_list

        conexao_socket.send(pickle.dumps(self.bits_array))

        conexao_socket.close()
```

Estas duas funções são responsáveis por criar o socket e iniciar o servidor. A função `start_server` cria uma instância de `Thread` e a inicia. A função `_start_server` cria um socket, o associa ao endereço e porta fornecidos e o coloca em modo de escuta. Em seguida, ele entra em um loop infinito, aceitando conexões de entrada e recebendo dados. Os dados recebidos são desserializados e armazenados em `self.bits_array`. Em seguida, os dados são enviados de volta ao transmissor e a conexão é fechada.


Obs.: Se tentar rodar o código acima mais de uma vez, você receberá um erro, pois o socket ainda estará em uso. Para resolver isso, você pode reiniciar o kernel ou esperar alguns segundos até que o socket seja liberado.

## Método `run`

```py


def run(self, encoding_method, framing_method, error_correction_or_detection_method):

    match framing_method.lower():
        case "character_count":
            self.frames, self.padding_bits_list = self.character_count_deframing(
                self.bits_array)
        case "byte_insertion":
            self.frames, self.padding_bits_list = self.bytes_insertion_deframing(
                self.bits_array)
        case "bits_insertion":
            self.frames, self.padding_bits_list = self.bits_insertion_deframing(
                self.bits_array)

    match error_correction_or_detection_method.lower():
        case "even_parity":
            self.bits_cleaned, self.list_error_detec = self.solve_even_parity(
                self.frames, self.padding_bits_list)
        case "crc":
            self.bits_cleaned, self.list_error_detec = self.solve_crc32(
                self.frames, self.padding_bits_list)
        case "hamming":
            self.bits_cleaned, self.list_error_detec = self.solve_hamming(
                self.frames, self.padding_bits_list)

    match encoding_method.lower():
        case "nrz":  # -1 -> 0; 1 -> 1
            pass
        case "bipolar":  # 0 -> 0; (-1,1) -> 1
            pass
        case "manchester":  # 0 -> 0; 1 -> 1
            list_bits_cleaned = []
            for i in range(0, len(self.bits_cleaned), 2):
                if self.bits_cleaned[i] == 0 and self.bits_cleaned[i+1] == 1:
                    list_bits_cleaned = [0] + list_bits_cleaned
                elif self.bits_cleaned[i] == 1 and self.bits_cleaned[i+1] == 0:
                    list_bits_cleaned = [1] + list_bits_cleaned
            self.bits_cleaned = list_bits_cleaned

    final_str = self.__binary_2_text(self.bits_cleaned)

    bits_cleaned_str = ''.join(map(str, self.bits_cleaned))

    return self.bits_array, bits_cleaned_str, final_str

```

O método `run` da classe `Receiver` está processando uma sequência de bits recebida. Ele aplica um método de desenquadramento, um método de correção/detecção de erros e um método de decodificação, todos especificados pelos argumentos do método. Finalmente, ele converte a sequência de bits decodificada em texto e retorna a sequência de bits original, a sequência de bits decodificada como uma string e a string de texto decodificada.

Exemplo:

In [116]:
receptor.bits_cleaned = list("01001000011001010110110001101100011011110010000001010100010100100011000100100001") # Simulando a recepção de uma mensagem transmitida

simData = receptor.run("nrz", "character_count", "bit_parity")

print("Mensagem decodificada: ", simData[2])

Mensagem decodificada:  Hello TR1!


## Métodos de desenquadramento

```py


def character_count_deframing(self, bits_array):
        """Return a matrix of frames without headers"""
        original_frames_matrix = []
        padding_bits_list = []
        # bytes_list is a array of strings of 8 bits
        bytes_list = [''.join(map(str, bits_array[i:i+8]))
                      for i in range(0, len(bits_array), 8)]

        while bytes_list:
            # Convert the header to integer
            frame_size = int(bytes_list[0], 2)

            padding_bits = int(bytes_list[1], 2)
            # Remove the header and add the frame to the matrix
            original_frames_matrix.append(bytes_list[2:frame_size])
            padding_bits_list.append(padding_bits)
            # Move to the next frame
            bytes_list = bytes_list[frame_size:]

        list_str_frames = [''.join(frame) for frame in original_frames_matrix]

        return list_str_frames, padding_bits_list
```

O método `character_count_deframing` desenquadra a sequência de bits recebida usando o método de contagem de caracteres. Ele retorna uma lista de quadros sem cabeçalhos e uma lista de bits de preenchimento.

Exemplo:

In [117]:
example = list(map(int, list("01001000") + list("01100101") + list("01101100") + list("01101100") + list("01101111")))

print("Mensagem Recebida:", example)

print("Desenquadramento Cont. Carac.:", receptor.character_count_deframing(example))

Mensagem Recebida: [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
Desenquadramento Cont. Carac.: (['011011000110110001101111'], [101])


```py

def bytes_insertion_deframing(self, bits_array):
    """Return a matrix of frames without flags"""
    original_frames_matrix = []
    frame = []
    padding_bits_list = []
    # bytes_list is a array of strings of 8 bits
    bytes_list = [''.join(map(str, bits_array[i:i+8]))
                    for i in range(0, len(bits_array), 8)]
    flag = '01111110'

    while bytes_list:
        byte = bytes_list.pop(0)
        if byte == flag:
            if frame:  # if frame is not empty
                original_frames_matrix.append(frame)
                padding_bits = int(frame[0], 2)
                padding_bits_list.append(padding_bits)
                frame = []
        else:
            frame.append(byte)

    list_str_frames = [''.join(frame) for frame in original_frames_matrix]

    return list_str_frames, padding_bits_list
```

O método `bytes_insertion_deframing` desenquadra a sequência de bits recebida usando o método de inserção de bytes.
Neste método, os quadros são delimitados por uma flag de 8 bits, que é 01111110. O método retorna uma lista de quadros sem flags e uma lista de bits de preenchimento.

Exemplo:


```py
def bits_insertion_deframing(self, bits_array, crc32=False):
    """Return a list of frames(str) without flags"""
    original_frames_list = []
    frame = []
    padding_bits_list = []
    # bits_string is a string of bits
    bits_string = ''.join(map(str, bits_array))
    flag = '01111110'

    while bits_string:
        if bits_string.startswith(flag):
            bits_string = bits_string[len(flag):]  # remove the flag
            if crc32:
                padding_bits = int(bits_string[:8], 2)
                bits_string = bits_string[8:]
                padding_bits_list.append(padding_bits)

            if frame:  # if frame is not empty
                original_frames_list.append("".join(frame))
                frame = []
        else:
            frame.append(bits_string[0])
            bits_string = bits_string[1:]

    return original_frames_list, padding_bits_list

```

O método `bits_insertion_deframing` desenquadra a sequência de bits recebida usando o método de inserção de bits.
Neste método, os quadros são delimitados por uma flag de 8 bits, que é 01111110. O método retorna uma lista de quadros sem flags e uma lista de bits de preenchimento.

Exemplo:

In [118]:
example = "01111110011110010110000101101110101111110"

print("Mensagem Recebida:", example)

print("Desenquadramento Inserção de Bits:", receptor.bits_insertion_deframing(example))

Mensagem Recebida: 01111110011110010110000101101110101111110
Desenquadramento Inserção de Bits: (['0111100101100001011011101'], [0])


Note que o método recebe uma flag de 8 bits, mas o CRC32 usa uma flag de 32 bits. Para resolver isso, o método verifica se o argumento `crc32` é `True` e, se for, ele remove os 8 bits de preenchimento do início da sequência de bits e os armazena em `padding_bits_list`.

```py


if crc32:
    padding_bits = int(bits_string[:8], 2)
    bits_string = bits_string[8:]
    padding_bits_list.append(padding_bits)
```

Exemplo:


In [119]:
example = "01111110011110010110000101101110101111110"

print("Mensagem Recebida:", example)

print("Desenquadramento Inserção de Bits (CRC32):", receptor.bits_insertion_deframing(example, True))

Mensagem Recebida: 01111110011110010110000101101110101111110
Desenquadramento Inserção de Bits (CRC32): (['01100001011011101'], [121])


## Métodos de correção/detecção de erros

```py


def solve_even_parity(self, frames, padding_bits_list):
    list_detection_error = []
    list_bits_cleaned = []
    if padding_bits_list:
        for frame, padding_bits in zip(frames, padding_bits_list):
            if padding_bits != 0:
                frame = frame[:-padding_bits]  # remove the padding bits

            # convert the frame to a list of integers
            bits_array = [int(bit) for bit in frame]
            count_ones = sum(bits_array[:-1])
            parity_bit = bits_array[-1]

            if (count_ones + parity_bit) % 2 == 0:
                list_detection_error.append(False)
            else:
                list_detection_error.append(True)

            list_bits_cleaned.extend(bits_array[:-1])

    else:
        for frame in frames:
            bits_array = [int(bit) for bit in frame]
            count_ones = sum(bits_array[:-1])
            parity_bit = bits_array[-1]

            if (count_ones + parity_bit) % 2 == 0:
                list_detection_error.append(False)

            else:
                list_detection_error.append(True)

            list_bits_cleaned.extend(bits_array[:-1])

    return list_bits_cleaned, list_detection_error
```

Este método verifica a paridade par de cada quadro em uma lista de quadros. 

Ele percorre cada quadro e, se houver bits de preenchimento, os remove. Em seguida, conta o número de uns no quadro (excluindo o bit de paridade) e verifica se a soma do número de uns e o bit de paridade é par. Se for, adiciona `False` à `list_detection_error`, indicando que não há erro de paridade. Se não for, adiciona `True`, indicando um erro de paridade. 

Os bits do quadro (excluindo o bit de paridade) são adicionados à `list_bits_cleaned`. 

O método retorna a `list_bits_cleaned` e a `list_detection_error`.

Exemplo:

In [120]:
frames = ['1011001', '1000111', '1111000']
padding_bits_list = [0, 1, 0]
bits_cleaned, detection_error = receptor.solve_even_parity(frames, padding_bits_list)

print(bits_cleaned) 
print(detection_error)  

[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
[False, True, False]


```py

def solve_crc32(self, frames, padding_bits_list):
    def verify_crc32(bit_array):
        inserted_bits_len = 0
        # polynomial used by CRC32 IEEE 802 (0x04C11DB7 without the occlusion of the first bit)
        crc32_polynomial = 0x104C11DB7
        # 32 0's to complete the 96 bits
        crc32_polynomial_str = f"{crc32_polynomial:033b}"

        def xor(bit_str_a, bit_str_b):  # xor between two bit strings
            bit_str_result = ''
            for i in range(len(bit_str_a)):
                # if the bits are equal, the result is 0
                if bit_str_a[i] == bit_str_b[i]:
                    bit_str_result += '0'
                else:  # if the bits are different, the result is 1
                    bit_str_result += '1'
            return bit_str_result

        # convert the bit array to a bit string
        bit_str = ''.join(map(str, bit_array))

        bit_str = bit_str + '0'*32  # 32 0's to complete the 96 bits
        bit_str_to_xor = ''
        for i in range(len(bit_str)):  # for each bit in the bit string
            if i <= 32:  # if the bit is in the first 32 bits
                bit_str_to_xor += bit_str[i]
            else:  # if the bit is in the last 64 bits
                if bit_str_to_xor[0] == '1':
                    # xor with the polynomial
                    bit_str_to_xor = xor(
                        bit_str_to_xor, crc32_polynomial_str)
                    # exclude the first bit (0) and include the next bit
                    bit_str_to_xor = bit_str_to_xor[1:] + bit_str[i]
                else:
                    # exclude the first bit (0) and include the next bit
                    bit_str_to_xor = bit_str_to_xor[1:] + bit_str[i]

            if i == len(bit_str) - 1:  # if it is the last bit
                # if the first bit is 1, xor with the polynomial
                if bit_str_to_xor[0] == '1':
                    bit_str_to_xor = xor(
                        bit_str_to_xor, crc32_polynomial_str)
                    # exclude the first bit (0)
                    bit_str_to_xor = bit_str_to_xor[1:]

                else:  # if the first bit is 0, xor with 33 0's
                    # exclude the first bit (0)
                    bit_str_to_xor = bit_str_to_xor[1:]

        if bit_str_to_xor == '0'*32:  # if the result is 32 0's, don't have errors
            return True
        else:  # if the result is not 32 0's, have errors
            return False

    list_detection_error = []
    list_bits_cleaned = []
    for frame, padding_bits in zip(frames, padding_bits_list):
        # convert the frame to a list of integers
        bits_array = [int(bit) for bit in frame]
        if verify_crc32(bits_array):
            list_detection_error.append(False)
        else:
            list_detection_error.append(True)

        if padding_bits != 0:
            # remove the padding bits
            list_bits_cleaned.extend(bits_array[:-padding_bits-32])
        else:
            list_bits_cleaned.extend(bits_array[:-32])

    return list_bits_cleaned, list_detection_error

```
Este método verifica a integridade dos quadros usando o algoritmo CRC32. 

Ele define uma função interna `verify_crc32` que realiza a verificação CRC32 em um array de bits. A função `xor` é usada para realizar a operação XOR entre duas strings de bits. 

Para cada quadro, ele converte o quadro em uma lista de inteiros e verifica se o CRC32 é válido. Se for, adiciona `False` à `list_detection_error`, indicando que não há erro. Se não for, adiciona `True`, indicando um erro. 

Os bits do quadro (excluindo os bits de preenchimento e os 32 bits do CRC) são adicionados à `list_bits_cleaned`. 

O método retorna a `list_bits_cleaned` e a `list_detection_error`.

In [121]:
frames = ['101100110100001011001000', '100011110000111000011110', '111100011110001111000111']
padding_bits_list = [0, 1, 0]
bits_cleaned, detection_error = receptor.solve_crc32(frames, padding_bits_list)

print(bits_cleaned)  
print(detection_error)  

[]
[True, True, True]


##  Solve Hamming

O método `solve_hamming` funciona da seguinte maneira:

`find_len_redundant_bits(bit_array)`: Esta função interna calcula o número de bits de paridade em um array de bits. Ela faz isso procurando o menor número `i` tal que `2**i` seja maior ou igual ao tamanho do array de bits.

`calculate_parity_bit(bit_array, position)`: Esta função interna calcula o valor do bit de paridade para uma determinada posição. Ela faz isso coletando todos os bits que estão em posições que são múltiplos da posição do bit de paridade, e então fazendo um XOR de todos esses bits.

`make_correction(bit_array)`: Esta função interna verifica os bits de paridade e usa-os para detectar e corrigir erros. Ela também remove os bits de paridade do array de bits.

A função `solve_hamming` então aplica a função `make_correction` a cada quadro em `frames`, remove os bits de preenchimento (padding) e retorna os quadros decodificados e uma lista de erros detectados.